<a href="https://colab.research.google.com/github/svetoslavseo/google-ai-overview-checker/blob/main/AI_Overview_Checker.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
import requests
import json
import time
import csv

def load_keywords_from_csv(file_path):
    keywords = []
    with open(file_path, mode='r', encoding='utf-8') as file:
        reader = csv.reader(file)
        for row in reader:
            if row:  # Ensure the row is not empty
                keywords.append(row[0])
    return keywords

def get_serp_results(api_username, api_password, keywords, location_code=2826, language_code='en'):
    url = "https://api.dataforseo.com/v3/serp/google/organic/live/advanced"

    results = {}

    for keyword in keywords:
        payload = [{
            "keyword": keyword,
            "location_code": location_code,
            "language_code": language_code,
            "device": "mobile",
            "os": "android",
            "depth": 100,
            "load_async_ai_overview": True
        }]

        print(f"Sending request for: {keyword}")  # Debugging info
        print(json.dumps(payload, indent=4))

        response = requests.post(url, auth=(api_username, api_password), json=payload)

        if response.status_code == 200:
            data = response.json()
            results[keyword] = extract_ai_overview_references(data)
        else:
            print(f"Error fetching data for {keyword}: {response.status_code} {response.text}")
            results[keyword] = None

        time.sleep(2)  # Avoid hitting API rate limits

    return results

def extract_ai_overview_references(data):
    try:
        serp_data = data.get('tasks', [])[0].get('result', [])
        if serp_data:
            for result in serp_data[0].get('items', []):
                if result.get('type') == 'ai_overview':
                    references = result.get('references', [])
                    extracted_references = []
                    for ref in references:
                        extracted_references.append([
                            ref.get('source', ''),
                            ref.get('domain', ''),
                            ref.get('url', ''),
                            ref.get('title', ''),
                            ref.get('text', '')
                        ])
                    return extracted_references
        return []
    except Exception as e:
        print(f"Error parsing response: {e}")
        return []

def save_results_to_csv(results, output_file):
    with open(output_file, mode='w', newline='', encoding='utf-8') as file:
        writer = csv.writer(file)
        writer.writerow(["Keyword", "Source", "Domain", "URL", "Title", "Text"])
        for keyword, references in results.items():
            if references:
                for ref in references:
                    writer.writerow([keyword] + ref)
            else:
                writer.writerow([keyword, "No AI Overview", "", "", "", ""])

if __name__ == "__main__":
    api_username = "email" # paste you DataForSEO email here
    api_password = "password" # paste your pass here
    keywords_file = "/Users/petkovs/PycharmProjects/pythonProject/keywords - Sheet1.csv"  # Path to CSV file containing keywords
    output_file = "/Users/petkovs/PycharmProjects/pythonProject/ai_overview_results.csv"  # Output CSV file

    keywords = load_keywords_from_csv(keywords_file)

    results = get_serp_results(api_username, api_password, keywords)

    save_results_to_csv(results, output_file)

    print(f"Results saved to {output_file}")